In [1]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent, load_tools

from langchain_community.tools import ArxivQueryRun


In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2:2b", temperature=0.0)
llm



Ollama(model='gemma2:2b', temperature=0.0)

In [3]:
arxiv = ArxivAPIWrapper()
docs = arxiv.run("1605.08386")
docs

'Published: 2016-05-26\nTitle: Heat-bath random walks with Markov bases\nAuthors: Caprice Stanley, Tobias Windisch\nSummary: Graphs on lattice points are studied whose edges come from a finite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on\nfibers of a fixed integer matrix can be bounded from above by a constant. We\nthen study the mixing behaviour of heat-bath random walks on these graphs. We\nalso state explicit conditions on the set of moves so that the heat-bath random\nwalk, a generalization of the Glauber dynamics, is an expander in fixed\ndimension.'

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_community.llms import Ollama


tools = load_tools(
    ["arxiv"],
)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

k:\RAG Application\env\lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [7]:
agent_executor.invoke({"input":"write something about the research paper of Attention all you need"})


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to find the research paper "Attention is All You Need" and summarize it. 
Action: arxiv
Action Input: Attention is All You NeedPublished: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrocketed in recent months, and serving
models with low latencies remains challenging due to the quadratic input length
complexity of the attention layers. In this work, we investigate the effect of
dropping MLP and attention layers at inference time on the performance of
Llama-v2 models. We find that dropping dreeper attention layers only marginally
decreases performance but leads to the best speedups alongside dropping entire
layers. For example, removing 33\% of attention layers in a 13B Llama2 model
results in a 1.8\% drop in average performance over the OpenLLM benchmark. We
also observe that skip

{'input': 'write something about the research paper of Attention all you need',
 'output': ''}

In [14]:
# wikipedia searching using agents and wrapper classs
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# Load tools, including Wikipedia search tool
tools2 = load_tools(
    ["wikipedia"],
)

# Pull the prompt from the hub
custom_prompt = hub.pull("hwchase17/react").partial(
    sistema="""You are a helpful AI assistant tasked with finding information.
    Your goal is to find information about the topic and provide clear summaries.
    
    
    Guidelines:
    1. Be specific in your queries
    2. Limit to the requested 500 count of words
    
    """
)
# Create the React agent
agent2 = create_react_agent(llm, tools, custom_prompt)

# Create the agent executor
agent_executor2 = AgentExecutor(agent=agent2, tools=tools2,handle_parsing_errors=True, max_iterations=3,  # Limit the number of iterations
    early_stopping_method="force" ,verbose=True)


In [15]:
agent_executor2.invoke({
    "input":"what is meant by Loss Function and how it is helpful in deep learning network trainning"
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To understand loss function, I need to explain what a deep learning network does.  Then I can explain how the loss function helps in training. 
Action: wikipedia
Action Input: What is a deep learning network?Page: Deep learning
Summary: Deep learning is a subset of machine learning methods based on neural networks with representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields incl

{'input': 'what is meant by Loss Function and how it is helpful in deep learning network trainning',
 'output': 'Agent stopped due to iteration limit or time limit.'}